In [ ]:
import random
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from functools import reduce
import itertools as it
import seaborn as sns
from collections import Counter
import numpy
from matplotlib.pyplot import figure

In [ ]:
sns.set_style('darkgrid') # darkgrid, white grid, dark, white and ticks
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes
sns.color_palette('pastel')


In [ ]:
max_strat = 6
min_strat = -5
max_score = 5
min_score = -5

pairs = 125
generations = 150
print(generations)
population = 100
selection_bias = 100

cost = 0.1
benefit = 1

mutation = 0.001

simulations = 1000

In [ ]:
class Agent:
    def __init__(self, strategy, score, payoff, selection_bias):
        self.payoff = payoff
        self.strategy = strategy
        self.score = score
        self.selection_bias = selection_bias
        self.interactions = 0
    
    def make_child(self, parent):
        self.payoff = 0
        self.strategy = parent.strategy
        self.score = 0
        self.selection_bias = selection_bias
        self.interactions = 0
        
    def add_score(self, new_score):
        temp_score = new_score + self.score
        if temp_score >= min_score and temp_score <= max_score:
             self.score = temp_score
        elif temp_score <= min_score:
            self.score = min_score
        elif temp_score >= max_score:
            self.score = max_score

In [ ]:
def selectOne(population):
    max_value     = sum([c.payoff for c in population])
    pick    = random.uniform(0, max_value)
    current = 0
    for agent in population:
        current += agent.payoff
        if current >= pick:
                return agent
            
def selectOnePair(population1):
    population = population1.copy()
    i=0
    agents = []
    new_agent = random.sample(population,1)[0]
    agents.append(new_agent) 
    population.remove(new_agent)
    while i < 1:
        max_value = sum([c.selection_bias for c in population])
        pick    = random.uniform(0, max_value)
        current = 0
        for agent in population:
            current += agent.selection_bias
            if current > pick:
                agents.append(agent)
                population.remove(agent)
                break
        i += 1      
    return agents     

In [ ]:
def ProbabilityList(population_d):
    fitness = [c.payoff for c in population_d ]
    total_fit = sum(fitness)
    relative_fitness = [f/total_fit for f in fitness]
    return relative_fitness

def FitnessChoices(population, relative_fitness, number):
    return random.choices(population, weights = relative_fitness, k = number)

In [ ]:
def weird_division(n, d):
    return n / d if d else 0

mean_strategy_per_generation = []
mean_payoff_per_generation = []
global_strategies = []
initial_recipient_interactions_average = []
initial_donor_interactions_average = []
final_strategy_average = []
winner_list = []
power_law_list = []

for i in range(12):
    winner_list.append(0)

print(simulations)
for simulation in range(simulations) :
    
    amount_of_power_interactions = []
    
    for i in range(12):
        amount_of_power_interactions.append(0)

    if simulation % 50 == 0:
        print("simulation: ", simulation)
    agents = []
    power_strat = random.randint(min_strat, max_strat)
    for i in range(population):
        new_strat = random.randint(min_strat, max_strat)
        if new_strat == power_strat:
            agents.append(Agent(random.randint(min_strat, max_strat), 0,0, 1))
        else:
            agents.append(Agent(random.randint(min_strat, max_strat), 0,0, 1))
            
    

    for generation in range(generations) :
      #  if(generation % 1000 == 0):
            #   print(generation)
        for pair in range(pairs) :
            agent_pair = selectOnePair(agents)
            #print(len(agent_pair))
            donor = agent_pair[0]
            recipient = agent_pair[1]
            
            #donor.interactions += 1
            recipient.interactions += 1
            
            if recipient.score >= donor.strategy:
                
                recipient.payoff += benefit
                donor.add_score(benefit)
            else:
                donor.add_score(-benefit)

            recipient.payoff += cost
        
        if generation == 0:
            total_val = 0
            for agent in agents:
                total_val += agent.interactions * agent.strategy
                if agent.strategy >= 0:
                    amount_of_power_interactions[agent.strategy] += agent.interactions
                else:
                    amount_of_power_interactions[agent.strategy * -1 + max_strat] += agent.interactions
            initial_recipient_interactions_average.append(total_val / sum(agent.interactions for agent in agents))
            
    
        payoffs = []
        for agent in agents:
            payoffs.append(agent.payoff)

        if False:     
            if(generation % 500 == 0):

                points=[]
                interactions=[]
                payoffs = []

                for agent in agents:
                    payoffs.append(agent.payoff)
                    points.append(agent.strategy)
                    interactions.append(agent.interactions)

                plt.bar(points,interactions,align='center') # A bar chart
                plt.xlabel('Strategy')
                plt.ylabel('Interactions')
                plt.show()

                plt.bar(points,payoffs,align='center') # A bar chart
                plt.xlabel('Strategy')
                plt.ylabel('Payoff')
                plt.show()
        
        agents.sort(key=lambda x: x.payoff, reverse=True)
        for agent in range(len(agents)):       
            parent = selectOne(agents)
            agents[agent].make_child(parent)

        
        estrategias = []
        total = 0
        for agent in agents:
            estrategias.append(agent.strategy)
            global_strategies.append(agent.strategy)
            

            estrategias_numpy = numpy.array(estrategias)
        payoffs_numpy = numpy.array(payoffs)
        
        mean_strategy_per_generation.append(estrategias_numpy.mean())
        mean_payoff_per_generation.append(payoffs_numpy.mean())
        
    winner_strat = int(sum(estrategias) / population)
    if winner_strat >= 0:
        winner_list[winner_strat] += 1
    else:
        winner_list[winner_strat * -1 + max_strat] += 1    
        
    final_strategy_average.append(int(sum(a.strategy for a in agents) / population))
    initial_donor_interactions_average.append(amount_of_power_interactions[int(sum(a.strategy for a in agents) / population)])

for i in range(len(final_strategy_average)):
    if final_strategy_average[i] == 4:
        power_law_list.append(initial_recipient_interactions_average[i])

total_for_strats = []

for i in range(12):
    total_for_strats.append(0)

for i in range(simulations):
    if final_strategy_average[i] >= 0:
        total_for_strats[int(final_strategy_average[i])] = initial_recipient_interactions_average[i]
    else:
        total_for_strats[int(final_strategy_average[i])*-1 + max_strat] += initial_recipient_interactions_average[i]  
    
global_strategies_numpy = numpy.array(global_strategies)
plt.plot(mean_strategy_per_generation)
plt.show()

In [ ]:
sum(mean_strategy_per_generation)/len(mean_strategy_per_generation)


In [ ]:
def get_graph_with_color_and_size(width, height, dpi, data, background_color, palette_color, y_lim):
    figure(figsize=(width, height), dpi=dpi)
    ax = plt.gca()
    ax.set_facecolor(background_color)
    if y_lim > 0:
        ax.set_ylim([0, y_lim])

    plt.xlabel('Time (thousands of generations)',fontname='Helvetica')
    plt.ylabel('Strategy, k',fontname='Helvetica')
    plt.title('Average k for every generation',fontname='Helvetica')
    plt.plot(data, color=sns.color_palette('Set2')[palette_color])
    plt.show()

In [ ]:
figure(figsize=(20, 6), dpi=280)

plt.xlabel('Strategy, k',fontname='Helvetica')
plt.ylabel('Frequency',fontname='Helvetica')
ax = plt.gca()
ax.set_facecolor("#f2ffe6")

plt.title('Frequency distribution of strategies for {} generations'.format(generations),fontname='Helvetica')
x = range(0,generations)
print(Counter(global_strategies_numpy))
plt.hist( global_strategies_numpy, color=sns.color_palette('Set2')[0], linewidth=1, density=True)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from collections import Counter
%matplotlib inline
    
#df = pd.read_csv("./news_events_powerlaw.csv")
#df["bucket"] = df["bucket"] + 1

c = Counter(power_law_list)
c1 = dict(c)
r=[]
for l in c1.values():
    r.append(l/sum(c1.values()))

f, ax = plt.subplots(figsize=(10, 10))
ax.set(xscale="linear", yscale="linear")
ax.scatter(c1.keys(),r)

ax.set_title('Distribution')
ax.set_xlabel('Interaction Average')
ax.set_ylabel('PDF')
#plt.savefig("./figures/scatter.png")

#print df.head()
#print df.tail()

In [ ]:
figure(figsize=(10, 5), dpi=80)    
    
plt.xlabel('Winning strategy, k',fontname='Helvetica')
plt.ylabel('Average interactions for 1st generation',fontname='Helvetica')
plt.scatter(x=final_strategy_average, y=initial_recipient_interactions_average, s=300, alpha=0.15)
plt.show()

print(str(total_for_strats) + "    " + str(winner_list))        
        
for i in range(min_strat, max_strat):
    if i >= 0:
        val = weird_division(total_for_strats[i], winner_list[i])
    else:
        val = weird_division(total_for_strats[i*-1 + max_strat], winner_list[i * -1 + max_strat])
    print("Average interactions for k=" + str(i) + " -> " + str(val))

figure(figsize=(10, 10), dpi=80)    

plt.xlabel('Winning Strategy, k',fontname='Helvetica')
plt.ylabel('Number of receiver interactions for 1st generation',fontname='Helvetica')
plt.scatter(x=final_strategy_average, y=initial_donor_interactions_average, s=300, alpha=0.15)
plt.show()